## Cycle extraction

Testing cycle extraction by iterating on cycle of extraction+subtraction

In [1]:
using  PyCall
using  Statistics , StatsBase
using  DataFrames, Printf , CSV , Query
using  Dates, Glob

import PyPlot
sns= pyimport("seaborn")

## directory
rootdir = ENV["GAIA_ROOT"]
wdir    = "$rootdir/products"
votdir  = "$rootdir/e2e_products/votable.2020"
plotdir = "$wdir/test"
ocdir   = "$wdir/octest"
sclist  = "$rootdir/e2e_products/sc-list-2020.csv"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

cd(wdir)

┌ Info: Precompiling GaiaClustering [top-level]
└ @ Base loading.jl:1260


In [2]:
function _getdata(filevot)
    voname = filevot

    println("## Data filtered in distance ..")
    data       = read_votable(voname)
    df         = filter_data(data,[0., 2000])
    dfcart     = add_cartesian(df)

    return(df, dfcart)
end

_getdata (generic function with 1 method)

In [3]:
function _remove_stars(df, dfcart, idx)
    # println(size(df.data))
    # println("setdiff")
    # println(size(df.data[:,setdiff(1:end,idx)]))
    s=size(df.data)
    # println(size(df.data[:,setdiff(1:s[2],idx)]))
    diff= setdiff(1:s[2],idx)
    # println(length(diff))
    
    #println("end setdiff")
    
    dfdata= df.data[:,setdiff(1:end,idx)]
    dfraw= df.raw[:,setdiff(1:end,idx)]
    dferr= df.err[:,setdiff(1:end,idx)]
    
    dfcartdata= dfcart.data[:,setdiff(1:end,idx)]
    dfcartraw= dfcart.raw[:,setdiff(1:end,idx)]
    dfcarterr= dfcart.err[:,setdiff(1:end,idx)]
     
    s=size(dfdata)
    # println("##ndata")
    # println(s[2])
    # println("##ndata")
    
    dfnew= GaiaClustering.Df(s[2],dfdata,dfraw,dferr)
    dfcartnew= GaiaClustering.Df(s[2],dfcartdata,dfcartraw,dfcarterr)
    
    # println(size(dfnew.data))
    nrem= length(idx)
    println("### $nrem stars removed")
    return(dfnew, dfcartnew)
end

function _extraction_mcmc(mc, votname)
    epsm = median(mc.eps)
    epsd = std(mc.eps)
    mneim = median(mc.mne)
    mneid = std(mc.mne)
    mclm = median(mc.mcl)
    mcld = std(mc.mcl)
    qcm = median(mc.qc)
    qnm = median(mc.qn)
    qcd = std(mc.qc)
    qnd = std(mc.qn)
    w3dm = median(mc.w3d)
    w3dd = std(mc.w3d)
    wvelm = median(mc.wvel)
    wveld = std(mc.wvel)
    whrdm = median(mc.whrd)
    whrdd = std(mc.whrd)
    
    @printf("## DBSCAN/MCMC stats: \n")
    @printf("### ϵ : %3.3f +/- %3.3f \n", epsm, epsd)
    @printf("### min_nei  : %3.1f +/- %3.3f \n", mneim, mneid)
    @printf("### min_clus  : %3.1f +/- %3.3f \n", mclm, mcld)
    @printf("### W3d  : %3.3f +/- %3.3f \n", w3dm, w3dd)
    @printf("### Wvel  : %3.3f +/- %3.3f \n" , wvelm, wveld)
    @printf("### Whrd  : %3.3f +/- %3.3f \n", whrdm, whrdd)
    @printf("### Qn  : %3.3f +/- %3.3f \n",qnm, qnd)
    @printf("### Qc  : %3.3f +/- %3.3f \n",qcm, qcd)
    @printf("##")
    
    res = DataFrame(votname=votname, epsm = epsm, epsd=epsd, mneim=mneim,mneid=mneid,mclm=mclm,mcld=mcld,
            qcm=qcm,qcd=qcd, qnm=qnm,qnd=qnd,
            w3dm=w3dm,w3dd=w3dd,wvelm=wvelm,wveld=wveld,whrdm=whrdm,whrdd=whrdd)
    return(res)
end

function _score_cycle(qc, qn, nchain, iter)
    k= log10(qc*qn*nchain /iter)
    
    return(k)
end

_score_cycle (generic function with 1 method)

In [4]:
function _plot_check(dfcart)
    cart= DataFrame(X=dfcart.data[1,:], Y=dfcart.data[2,:], Z=dfcart.data[3,:])

    println("## check plot subtraction ...")
    
    PyPlot.plt.figure(figsize=(9.0,8.0))
    PyPlot.plt.subplot(1, 1, 1 , ylim=[100,450], xlim=[-50,50])
    PyPlot.plt.scatter(cart.Y, cart.X, s = 0.1 )
    PyPlot.plt.xlabel("Y (pc)")
    PyPlot.plt.ylabel("X (pc)")
    PyPlot.plt.grid(true)
    
    PyPlot.plt.show()

end

_plot_check (generic function with 1 method)

In [5]:
## update only the fileres with DF
function _updt!(fileres, df)
    if !isfile(fileres)
        CSV.write(fileres,df,delim=';')
        println("## $fileres created...")
        return(true)
    else
        res = DataFrames.copy(CSV.read(fileres, delim=";"))
        append!(res,df)
        CSV.write(fileres,res,delim=';')
        return(true)
    end
end

_updt! (generic function with 1 method)

In [6]:
## main cycle method
## filedebug is mainly to control the different time lapse
##
function _cycle_extraction(votname, filedebug, m::GaiaClustering.meta)
    println("############### extraction #########")
    df, dfcart= _getdata(votdir*"/"*votname)
    
    cyclerun= true
    cycle= 1
    
    cyclemax= m.cyclemax
    minstarselection=   m.minstarselection    # minimum of stars to select solution in a cycle...????
    minstarstop=   m.minstarstop         # condition to stop cycling
    minchainreached=  m.minchainreached      # minimum chain to analyze solution
    qcmin=  m.qcmin                # more condition on Qc to stop cycling after the first
    wratiomin=  m.wratiomin          # minimum ratio btwn w3d and wvel (otherwise not an OC)
    
    #if isfile(filedebug)
    #    println("## $filedebug removed...")
    #    rm(filedebug)
    #end
    
    println("##")
    while cyclerun
        tstart= now()
        println("##\n## starting cycle $cycle ...")
        ## extraction one cycle.. MCMC optimization
        mc , iter, flag= abc_mcmc_dbscan_full2(dfcart, m)
        println("## Flag: $flag")
        nchain= length(mc.qc)
        println("## $iter iterations performed...")
        println("## $nchain chains")
        
        if flag== -1 || nchain > minchainreached
            println("## optimization completed..")
            println("## analyzing solutions...")
            plot_dbscanfull_mcmc(plotdir, "$votname.$cycle", mc , false)

            ## get the cluster and plot it
            println("## extracting the cluster using DBSCAN/WEIGHTING with:")
            res2= _extraction_mcmc(mc, votname)
            eps= res2.epsm[1]
            min_nei= trunc(Int,res2.mneim[1] + 0.5)
            min_cl= trunc(Int,res2.mclm[1] + 0.5)
            w3d= res2.w3dm[1]
            wvel= res2.wvelm[1]
            whrd= res2.whrdm[1]

            mres = GaiaClustering.modelfull(eps,min_nei,min_cl,w3d,wvel,whrd)
            dfcartnorm = getDfcartnorm(dfcart, mres)
            labels = clusters(dfcartnorm.data ,eps  , 20, min_nei, min_cl)
            labelmax , nmax, qc = find_cluster_label2(labels, df, dfcart)
            println("## label $labelmax written to oc...")
            export_df("$votname.$cycle", ocdir, df , dfcart, labels , labelmax)
            scproperties0 = get_properties_SC(labels[labelmax] , df, dfcart)
            scproperties2 = get_properties_SC2(labels[labelmax] , df, dfcart)
        
            plot_cluster2(plotdir, "$votname.$cycle", labels[labelmax], scproperties0,  dfcart , false)
        
            println("###")
            println("### Label solution: $labelmax")
            println("### N stars: $nmax")
            println("### Qc: $qc")
            println("###")
        
             _plot_check(dfcart)
        
            k= _score_cycle(qc, nmax, nchain, iter)
            @printf("## score cycle %d: %3.3f \n",cycle, k)
                
            println("###")
            println("### subtracting BEST solution from Df...")
            dfnew, dfcartnew= _remove_stars(df, dfcart, labels[labelmax])
            df= dfnew
            dfcart= dfcartnew
        
            ######################### STOP conditions
            if nmax < minstarstop
                println("## extraction stopped at cycle $cycle")
                println("## nmax too low...")
                cyclerun= false
            end
            if cycle == cyclemax
                println("## extraction stopped at cycle $cycle")
                println("## cyclemax reached...")
                cyclerun= false
            end
            if qc < qcmin
                println("## extraction stopped at cycle $cycle")
                println("## Qc too low...")
                cyclerun= false
            end
            if w3d/wvel < wratiomin || wvel/w3d < wratiomin
                println("## extraction stopped at cycle $cycle")
                println("## weight ratio too low...")
                cyclerun= false
            end
            if flag != -1
                println("## extraction stopped at cycle $cycle")
                println("## chain iteration not performed completely")
                cyclerun= false
            end
            ### 
            
            tend= now()
            duration= Dates.value(tend-tstart) / (1000*1)
            nstar= size(df.raw)[2]
            timeperiterstar= duration / (iter*nstar)
            timeperchainstar= duration / (nchain*nstar)
            @printf("## Time: \n")
            @printf("## duration per cycle %3.3f sec \n", duration)
            @printf("## duration per iteration*star %3.3e sec \n", timeperiterstar)
            @printf("## duration per chain*star %3.3e sec \n", timeperchainstar)
            @printf("##\n")
        
            ## log the results of performances
            dfout= DataFrame(votname=votname, cycle=cycle, nstar=nstar, qc=qc, nmax=nmax, nchain=nchain, iter=iter,
            scorecycle=k, duration=duration, timeperiterstar=timeperiterstar ,
            timeperchainstar= timeperchainstar )
            _updt!(filedebug, dfout)
            cycle += 1
        else
            println("## nothing found, stopped...")
            cyclerun= false
        end
    end
end

_cycle_extraction (generic function with 1 method)

In [7]:
function main(votlist, metafile, debugfile= "cycle_extraction.debug.csv")
    m= read_params(metafile)
    for votname in votlist
        println("## Analyzing $votname")
        _cycle_extraction(votname, debugfile, m)
    end
end

main (generic function with 2 methods)

In [ ]:
# votname= "NGC 869-3.0deg.vot"
metafile= "test.ext"
cd(votdir)
votlist= glob("NGC 110*vot")
cd(wdir)

main(votlist, metafile)

## All parameters set to default...
Any["minQc", "=", 2.7]
Any["minQn", "=", 40]
Any["maxQn", "=", 5000]
Any["forcedminstars", "=", 30]
Any["mingoodsolution", "=", 10]
Any["niterqminq", "=", 500]
Any["nburnout", "=", 1000]
Any["nchain", "=", 6000]
Any["maxiter", "=", 15000]
Any["cyclemax", "=", 4]
Any["minstarstop", "=", 50]
Any["minchainreached", "=", 100]
Any["qcmin", "=", 1.5]
Any["wratiomin", "=", 0.2]
Any["epsmean", "=", 2.5]
Any["epsdisp", "=", 1.5]
Any["min_nei", "=", 10]
Any["min_cl", "=", 15]
Any["ncoredisp", "=", 10]
Any["w3dmean", "=", 6.0]
Any["w3ddisp", "=", 4.0]
Any["wvelmean", "=", 6.0]
Any["wveldisp", "=", 4.0]
Any["whrdmean", "=", 2.0]
Any["whrddisp", "=", 1.5]
Any["aperture2d", "=", 1.5]
Any["maxaperture2d", "=", 15.0]
Any["aperture3d", "=", 3.0]
Any["maxaperture3d", "=", 20]
Any["aperturev", "=", 3.0]
Any["maxaperturev", "=", 20.0]
Any["nboot", "=", 50.0]
Any["labels", "=", "Qc"]
## Parameters read from test.ext
## Analyzing NGC 110-2.0deg.vot
############### extract

-- 55
modelfull(4.3763660048872275, 8, 18, 0.9822825918006437, 5.527883196879808, 4.149245927998249)
1.4130703255506677e-7
-- 56
modelfull(4.597172072536427, 11, 6, 11.20757737935718, 10.653498630481035, 2.4368726013193536)
1.0612605803673343e-7
-- 57
modelfull(0.80161045166417, 8, 13, 1.9732569696935665, 9.806026276105658, 0.7684859902714503)
2.817902173374426e-7
-- 58
modelfull(2.778565444591222, 3, 17, 7.446180015320216, 1.9215390766189469, 3.3324814317721843)
5.76295947772057e-7
-- 59
modelfull(2.926030471192789, 4, 5, 4.631850694074923, 8.325026321848085, 2.1123681781030315)
7.86838873864501e-7
-- 60
modelfull(2.2113158638856905, 16, 4, 8.322157882579845, 3.4480122654403966, 2.533583886879323)
5.897893923190372e-7
-- 61
modelfull(4.3606374217145065, 20, 13, 1.5449094360727793, 7.638060673025568, 1.1710338551952266)
2.3804427572919402e-7
-- 62
modelfull(1.3954894938523017, 18, 25, 2.9641874642195267, 13.232566584644365, 2.286779192950359)
9.815225518135917e-8
-- 63
modelfull(3.6821

-- 122
modelfull(3.2359960855916787, 19, 14, 6.9436641826613945, 4.460412534329472, 3.060111446043223)
8.426650556586973e-7
-- 123
modelfull(0.19405702376387213, 4, 2, 7.339994569091585, 5.685924223257413, 1.059259432992666)
1.734846935071241e-7
-- 124
modelfull(1.5888812258373792, 3, 10, 3.3731626654576816, 6.117586191338935, 0.8528892504454353)
7.031940579307714e-7
-- 125
modelfull(1.8103818704784622, 25, 22, 15.631783129085232, 6.117349635674602, 0.5957699529863527)
1.6530072107403926e-8
-- 126
modelfull(2.943102644395488, 30, 18, 10.991690828710457, 4.74636855003305, 3.8156590518332036)
5.296370827280731e-8
-- 127
modelfull(2.7506878118655202, 3, 19, 6.351282405314977, 11.500128033778251, 1.8463888128717179)
5.584503664209966e-7
-- 128
modelfull(4.413916913649599, 20, 28, 11.184480565254091, 5.283638091060039, 3.8269922742715883)
4.7551535852605016e-8
-- 129
modelfull(2.786290289767484, 2, 6, 5.901302968166673, 5.427768101502318, 1.2460598012468533)
8.44309969580385e-7
-- 130
model

-- 189
modelfull(4.1068275903624665, 10, 7, 5.807960836085661, 3.5022827726434604, 1.9503950917648136)
6.841504599538955e-7
-- 190
modelfull(4.158620063448491, 28, 22, 1.974866412601398, 7.063905990426601, 1.914627911196333)
9.938042138677361e-8
-- 191
modelfull(1.247515538050458, 23, 31, 10.235581384844144, 10.763419924110353, 2.7594292013534765)
4.239894170430615e-8
-- 192
modelfull(3.0213836858657297, 27, 19, 5.811493361263267, 2.2511638417605044, 2.1608333892666685)
2.6697079986364565e-7
-- 193
modelfull(1.1678912414649572, 17, 13, 3.18683955663048, 5.246127160743257, 0.13506441760359866)
3.728950609173225e-7
-- 194
modelfull(2.2181485149738154, 4, 20, 6.42976793139958, 2.6093905167363065, 1.6259626436085306)
9.920913904255622e-7
-- 195
modelfull(4.863052735633019, 4, 17, 7.934201784065954, 0.2948414268697954, 4.223412783464033)
5.1679601530967644e-8
-- 196
modelfull(4.43781842099871, 7, 9, 2.701675394722704, 6.174270248146029, 3.666906947578878)
2.706249427416695e-7
-- 197
modelfu

-- 256
modelfull(1.5256134203351563, 1, 24, 3.8468426120308776, 8.483652162410625, 3.039121321626597)
4.115824993286093e-7
-- 257
modelfull(5.26078644596962, 2, 21, 8.073427875596778, 6.66827328562643, 4.324335145860697)
5.890681185486293e-8
-- 258
modelfull(0.8240147136386446, 5, 22, 5.591384809053641, 4.345812514553515, 3.2913407577617946)
4.748905286388043e-7
-- 259
modelfull(1.98455070143522, 6, 23, 9.428281168309324, 4.537263036446634, 2.145278435351161)
8.294048260303747e-7
-- 260
modelfull(3.306401487430315, 20, 6, 8.303376749724148, 2.9175852264606714, 2.6802078742324147)
4.048628852555394e-7
-- 261
modelfull(1.2665824482061536, 19, 27, 2.3537543325624273, 4.6558582130875035, 4.002499286467931)
1.2059921876711444e-7
-- 262
modelfull(1.088925993884893, 13, 19, 7.479520655187146, 9.365795025123228, 1.9133104114334307)
7.552286537484081e-7
-- 263
modelfull(5.732701510144549, 20, 23, 8.407540932524732, 4.9218266091180904, 1.3352885037612026)
6.411246704597548e-8
-- 264
modelfull(1.

...

-- 408
modelfull(4.385149029824705, 6, 23, 10.65801525678359, 9.910478255525057, 2.860955287937217)
1.6537765390038003e-7
-- 409
modelfull(3.7666497792708546, 16, 11, 7.34838738943577, 3.097288509464223, 0.49496374583204306)
4.823014703715214e-7
-- 410
modelfull(4.693751006382108, 31, 16, 6.7649224288302365, 10.844445256543782, 2.8948492367423904)
3.0250356538901244e-8
-- 411
modelfull(2.1256034346316808, 5, 13, 4.529509525898404, 7.393910732562237, 0.5975958857430297)
9.698037807629147e-7
-- 412
modelfull(4.145636018979987, 22, 12, 3.1310778800652224, 6.7910901593804756, 4.232872172112085)
1.2994085678443514e-7
-- 413
modelfull(3.2212146046712418, 22, 10, 10.735182888738663, 2.442763089164529, 2.2336300057294696)
2.5717258282678236e-7
-- 414
modelfull(3.2393744449513355, 21, 16, 3.2255450626296773, 5.1710893491393195, 1.6614852257399373)
7.348142208405167e-7
-- 415
modelfull(2.616418878570412, 8, 3, 4.194705117944531, 4.371994182746997, 3.198967745663444)
5.849078698682885e-7
-- 416
m

-- 475
modelfull(3.669677159688889, 13, 16, 10.424237246556212, 11.010108848062323, 2.0737759740606587)
3.532761551957213e-7
-- 476
modelfull(3.797227045345199, 13, 7, 3.124182518712306, 2.8386835385679414, 2.8401007502376823)
4.6967317034851965e-7
-- 477
modelfull(1.5612968679328645, 18, 4, 3.0533791623608715, 4.453677669851968, 2.332370169087278)
4.5813863073265894e-7
-- 478
modelfull(3.435073176307646, 8, 19, 8.421787762143971, 8.46003427128379, 1.041054061911968)
8.519549359514626e-7
-- 479
modelfull(7.418917819976741, 6, 12, 7.875851202335959, 5.685731048619673, 4.345165353832053)
2.184804383687835e-9
-- 480
modelfull(2.3627203130948398, 4, 16, 10.632967032142645, 1.6711259944087464, 0.5054012640800993)
2.9196488661816927e-7
-- 481
modelfull(1.4302076427079597, 4, 26, 7.672047831880208, 12.189871196925171, 0.7753035415043699)
1.427738324335531e-7
-- 482
modelfull(2.116011490292082, 22, 34, 11.258657328167061, 3.4040379681418607, 2.3067602905650455)
5.272788737984731e-8
-- 483
mode

-- 42
modelfull(3.076535761711977, 10, 21, 0.23542248302292812, 14.508866802579606, 2.861857247716194)
4.933738542283457e-8
-- 43
modelfull(3.0478720992665354, 16, 3, 3.208765648437525, 12.084860730138741, 0.33190206494636465)
1.0282449877137827e-7
-- 44
modelfull(3.3963504629746035, 5, 19, 1.3133077852319728, 1.9862903334158561, 1.622325952503056)
4.089140455418928e-7
-- 45
modelfull(3.437465715706094, 5, 14, 7.2836934490922145, 5.613031677785854, 2.8735073003399565)
1.1732758947551198e-6
-- 46
modelfull(2.113586269824359, 17, 20, 11.955400202339572, 6.74262338110951, 0.824391461447608)
3.246382860012342e-7
-- 47
modelfull(0.793280327170141, 21, 17, 6.8949300246606855, 2.69962196963724, 2.0213013852168458)
3.957405348456209e-7
-- 48
modelfull(2.869177970896146, 2, 16, 7.674917681602499, 8.640554963192244, 3.81084966557445)
5.072661008138418e-7
-- 49
modelfull(1.9723435903479485, 16, 26, 7.061048451838696, 10.08316089645, 1.0076804092117129)
4.021020852542732e-7
-- 50
modelfull(1.96332

-- 109
modelfull(4.605974378229987, 4, 2, 9.352598893350489, 4.763207864047737, 0.9902233061668673)
1.458189345287504e-7
-- 110
modelfull(2.1939096286294766, 25, 8, 0.4916523678608211, 4.279267267332546, 2.4644545362772696)
1.7056508542900467e-7
-- 111
modelfull(3.7511526483463316, 13, 22, 7.662385141315851, 3.670033005523233, 1.9095044741994494)
8.312215281242706e-7
-- 112
modelfull(1.415529829710827, 19, 15, 5.302546782567163, 4.7790363973810885, 0.5426125408599114)
6.13055201767607e-7
-- 113
modelfull(3.6142536675458237, 5, 8, 8.244208347955889, 6.219760299007014, 2.557324427480999)
8.495965274845245e-7
-- 114
modelfull(1.7697329745644133, 15, 4, 10.344234179931343, 10.494759304449714, 1.7275265356404557)
2.527698788521093e-7
-- 115
modelfull(1.6807637862739981, 15, 4, 4.78445695502697, 2.4522165062284267, 0.8593205116712714)
4.0780511766858165e-7
-- 116
modelfull(1.6086604219752583, 14, 21, 5.19587588947718, 4.820931059860374, 4.1481602927585675)
4.427119880219604e-7
-- 117
modelfu

-- 177
modelfull(1.50042145103977, 6, 12, 7.446161992252159, 8.140640516542517, 0.19095295311856408)
5.643884883283192e-7
-- 178
modelfull(3.0911147701423607, 21, 31, 3.9279808879238525, 4.563470966678815, 3.4757159487354463)
1.445095006050933e-7
-- 179
modelfull(1.5556950947765185, 14, 9, 9.963882773042467, 7.285355718297552, 3.000759053112464)
5.989803910966317e-7
-- 180
modelfull(3.8845724007379565, 21, 8, 4.224185004622764, 5.153367154947939, 2.642465480026197)
4.5939323569519474e-7
-- 181
modelfull(3.5292518099192254, 4, 26, 7.069413002118907, 2.6863175001658686, 2.227896306369534)
4.96584485633097e-7
-- 182
modelfull(2.2181600679373012, 14, 7, 5.634441442850124, 3.4523325952387496, 1.3633999270660961)
9.961225899831577e-7
-- 183
modelfull(2.7119814694204845, 14, 8, 15.280817405136764, 16.61097849260335, 2.2886432098704557)
2.8716482883877275e-9
-- 184
modelfull(4.026712511281293, 11, 25, 4.906544142388183, 5.784766437232682, 2.1559523793811977)
7.002268398089823e-7
-- 185
modelfu